In [ ]:
from google.cloud import bigquery as bq

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
conn = bq.Client.from_service_account_json("case.json")

# Dimensions Exploration

#### What?
We explore key **categorical dimensions** across the game’s core datasets:
- `platform`, `network`, and `country` from **install data**
- `status` from **level completion data**
- `package_type` from **revenue data**
- `network` from **cost data**

These dimensions represent non-numeric attributes that define how users interact with the app and how the app performs across various channels and markets.

---

#### Why?
Understanding the variety and distribution of these dimensions helps to:
- **Identify segmentation opportunities** (e.g., by platform or country)
- **Ensure data coverage** across all relevant user groups and monetization channels
- **Prepare for deeper analysis**, such as user behavior by network or revenue by package type

---

#### How?
We use `SELECT DISTINCT` or `ARRAY_AGG(DISTINCT ...)` queries to extract **unique values** from each dimension. These operations allow us to:
- List out all existing categories (e.g., all platforms or package types)
- Assess the **diversity** and **granularity** in the data
- Facilitate grouping and filtering in future analysis steps


In [ ]:
query_de_1 = """
-- Retrieve distinct values of platform, network, and country from the install data
-- Aggregates these values into arrays for easy reference and comparison

SELECT
  ARRAY_AGG(DISTINCT platform) AS unique_platforms,    -- Collects all unique platforms into a single array
  ARRAY_AGG(DISTINCT network) AS unique_networks,      -- Collects all unique networks into a single array
  ARRAY_AGG(DISTINCT country) AS unique_countries      -- Collects all unique countries into a single array
FROM `case_db.q1_table_install`;
"""

In [ ]:
 df_de_1 = conn.query(query_de_1).to_dataframe()

In [ ]:
df_de_1

,unique_platforms,unique_networks,unique_countries
0,"[ios, android]","[Sid, Buzz, Woody, Jessie, Organic]","[Venus, Pluton, Saturn, Uranus, Mercury]"


In [ ]:
query_de_2 = """
-- Retrieve all unique status values from level completion data

SELECT
   DISTINCT status AS unique_statuses  -- Extracts each distinct status (e.g., success, failure)
FROM `case_db.q1_table_level_end`;
"""

In [ ]:
 df_de_2 = conn.query(query_de_2).to_dataframe()

In [ ]:
df_de_2

,unique_statuses
0,win
1,fail
2,quit


In [ ]:
query_de_3 = """
-- Retrieve all unique package types from the revenue data

SELECT
  DISTINCT package_type AS unique_package_types  -- Identifies each distinct in-app purchase or subscription type
FROM `case_db.q1_table_revenue`;

"""

In [ ]:
 df_de_3 = conn.query(query_de_3).to_dataframe()

In [ ]:
df_de_3

,unique_package_types
0,awesome_packs
1,lovely_packs
2,pretty_packs
3,cool_packs


In [ ]:
query_de_4 = """
-- Retrieve all unique ad networks from the cost data

SELECT
  DISTINCT network AS unique_networks  -- Extracts distinct advertising networks used
FROM `case_db.q1_table_cost`;

"""

In [ ]:
 df_de_4 = conn.query(query_de_4).to_dataframe()

In [ ]:
df_de_4

,unique_networks
0,Sid
1,Buzz
2,Woody
3,Jessie
4,Organic


# Date Exploration

#### What?
We examine the **temporal coverage** of the main datasets in the `case_db` project by identifying:
- The **earliest (`min_date`)** and **latest (`max_date`)** event timestamps
- The **date range** (in days) covered by each table

---

#### Why?
Understanding the date coverage helps to:
- **Validate data completeness** and detect any missing periods
- **Align datasets chronologically** for accurate analysis
- **Determine feasible time windows** for cohort, trend, or funnel analyses

For example, we ensure that install, session, and revenue data overlap, which is essential for conversion rate or LTV studies.

---

#### How?
We use SQL queries with `MIN()`, `MAX()`, and `DATE_DIFF()` functions to compute:
- The **start and end dates** for each table
- The **duration** of the available data in days

In [ ]:
query_dte_1 = """
-- Retrieve the minimum and maximum event dates, along with the total date range (in days),
-- for each key table in the `case_db` dataset to assess the temporal coverage of the data.

SELECT 'q1_table_install' AS table_name,
       DATE(MIN(event_time)) AS min_date,
       DATE(MAX(event_time)) AS max_date,
       DATE_DIFF(DATE(MAX(event_time)), DATE(MIN(event_time)), DAY) AS date_range_days
FROM `case_db.q1_table_install`

UNION ALL

SELECT 'q1_table_level_end' AS table_name,
       DATE(MIN(event_time)),
       DATE(MAX(event_time)),
       DATE_DIFF(DATE(MAX(event_time)), DATE(MIN(event_time)), DAY)
FROM `case_db.q1_table_level_end`

UNION ALL

SELECT 'q1_table_session' AS table_name,
       DATE(MIN(event_time)),
       DATE(MAX(event_time)),
       DATE_DIFF(DATE(MAX(event_time)), DATE(MIN(event_time)), DAY)
FROM `case_db.q1_table_session`

UNION ALL

SELECT 'q1_table_revenue' AS table_name,
       DATE(MIN(event_time)),
       DATE(MAX(event_time)),
       DATE_DIFF(DATE(MAX(event_time)), DATE(MIN(event_time)), DAY)
FROM `case_db.q1_table_revenue`

UNION ALL

SELECT 'q1_table_cost' AS table_name,
       MIN(DATE(date)),  -- `date` field is already a date; wrapped again for consistency
       MAX(DATE(date)),
       DATE_DIFF(MAX(DATE(date)), MIN(DATE(date)), DAY)
FROM `case_db.q1_table_cost`;
"""

In [ ]:
 df_dte_1 = conn.query(query_dte_1).to_dataframe()

In [ ]:
df_dte_1

,table_name,min_date,max_date,date_range_days
0,q1_table_revenue,2021-04-30,2021-06-14,45
1,q1_table_install,2021-04-30,2021-06-01,32
2,q1_table_cost,2021-05-01,2021-05-31,30
3,q1_table_session,2021-04-30,2021-06-14,45
4,q1_table_level_end,2021-04-30,2021-06-14,45


# Measures Exploration

#### What?
We explore **key quantitative measures** from the major tables in the dataset:
- Install metrics (e.g., installs, countries, platforms)
- Gameplay outcomes (e.g., wins, quits, level progress)
- Session activity (e.g., session count, time spent)
- Monetization (e.g., revenue, purchases)
- Marketing spend (e.g., ad costs by network/country/platform)

These **numerical KPIs** reflect user engagement, retention, monetization, and acquisition effectiveness.

---

#### Why?
Exploring measures allows us to:
- **Understand app performance** from different angles (user behavior, revenue, costs)
- **Establish baselines** for further KPIs like ARPU, ROAS, retention
- **Identify trends, anomalies, or optimization opportunities** (e.g., high spend but low installs on certain platforms)

Without these metrics, it would be impossible to judge the scale or efficiency of the product or marketing strategies.

---

#### How?
We use SQL aggregation functions such as:
- `COUNT()`, `SUM()`, `AVG()`, `MIN()`, `MAX()`
- `COUNTIF()` for conditional counts (e.g., wins, fails, quits)
- `CAST()` where necessary (e.g., revenue as `FLOAT64`)

In [ ]:
query_me_1 = """
-- Summarize key installation metrics from the install data:
-- Total users, total events, and diversity across country, network, and platform.

SELECT
  COUNT(DISTINCT user_id) AS total_installs,         -- Number of unique users who installed
  COUNT(*) AS total_install_events,                  -- Total install events (may include duplicates per user)
  COUNT(DISTINCT country) AS total_countries,        -- Number of unique countries where installs occurred
  COUNT(DISTINCT network) AS total_networks,         -- Number of unique ad networks driving installs
  COUNT(DISTINCT platform) AS total_platforms        -- Number of unique platforms (e.g., iOS, Android)
FROM `case_db.q1_table_install`;
"""

In [ ]:
 df_me_1 = conn.query(query_me_1).to_dataframe()

In [ ]:
df_me_1

,total_installs,total_install_events,total_countries,total_networks,total_platforms
0,215082,217415,5,5,2


In [ ]:
query_me_2 = """
-- Generate summary statistics from level completion data:
-- Includes user counts, level progression, outcomes, and player behavior metrics.

SELECT
  COUNT(*) AS total_level_ends,                          -- Total number of level end events
  COUNT(DISTINCT user_id) AS unique_players,             -- Number of unique users who completed levels
  COUNT(DISTINCT level) AS total_levels,                 -- Number of unique levels encountered
  MAX(level) AS max_level_reached,                       -- Highest level reached by any player
  MIN(level) AS min_level_reached,                       -- Lowest level encountered
  AVG(level) AS avg_level_reached,                       -- Average level number across all events
  COUNTIF(status = 'win') AS total_wins,                 -- Number of level completions with 'win' status
  COUNTIF(status = 'fail') AS total_fails,               -- Number of completions marked as 'fail'
  COUNTIF(status = 'quit') AS total_quits,               -- Number of sessions where players quit
  AVG(time_spent) AS avg_time_spent,                     -- Average time spent per level
  MAX(time_spent) AS max_time_spent,                     -- Maximum time spent on a level
  MIN(time_spent) AS min_time_spent,                     -- Minimum time spent on a level
  AVG(moves_made) AS avg_moves_made,                     -- Average number of moves made per level
  AVG(moves_left) AS avg_moves_left                      -- Average number of unused moves per level
FROM `case_db.q1_table_level_end`;
"""

In [ ]:
 df_me_2 = conn.query(query_me_2).to_dataframe()

In [ ]:
df_me_2

,total_level_ends,unique_players,total_levels,max_level_reached,min_level_reached,avg_level_reached,total_wins,total_fails,total_quits,avg_time_spent,max_time_spent,min_time_spent,avg_moves_made,avg_moves_left
0,72044374,203859,954,1010,1,229.842018,28698712,42494762,850900,98.477877,81712.0,1.0,29.779503,2.015845


In [ ]:
query_me_3 = """
SELECT
-- Summarize session activity data:
-- Includes total sessions, user count, session duration metrics, and level engagement.

SELECT
  COUNT(*) AS total_sessions,                             -- Total number of session events
  COUNT(DISTINCT user_id) AS unique_users,                -- Number of distinct users with session activity
  AVG(time_spent) AS avg_time_spent,                      -- Average time spent per session
  MAX(time_spent) AS max_time_spent,                      -- Longest individual session duration
  MIN(time_spent) AS min_time_spent,                      -- Shortest individual session duration
  COUNT(DISTINCT level) AS total_levels_touched           -- Number of unique levels visited during sessions
FROM `case_db.q1_table_session`;
"""

In [ ]:
 df_me_3 = conn.query(query_me_3).to_dataframe()

In [ ]:
df_me_3

,total_sessions,unique_users,avg_time_spent,max_time_spent,min_time_spent,total_levels_touched
0,297358858,214888,28.791623,155,0,958


In [ ]:
query_me_4 = """
-- Summarize purchase behavior and revenue statistics from in-app revenue data

SELECT
  COUNT(*) AS total_purchases,                               -- Total number of purchase events
  COUNT(DISTINCT user_id) AS paying_users,                   -- Number of distinct users who made purchases
  SUM(CAST(revenue AS FLOAT64)) AS total_revenue,            -- Total revenue generated
  MAX(CAST(revenue AS FLOAT64)) AS max_revenue,              -- Highest single purchase amount
  MIN(CAST(revenue AS FLOAT64)) AS min_revenue,              -- Lowest single purchase amount
  AVG(CAST(revenue AS FLOAT64)) AS avg_revenue,              -- Average revenue per purchase event
  COUNT(DISTINCT package_type) AS unique_package_types       -- Number of unique package types purchased
FROM `case_db.q1_table_revenue`;

"""

In [ ]:
 df_me_4 = conn.query(query_me_4).to_dataframe()

In [ ]:
df_me_4

,total_purchases,paying_users,total_revenue,max_revenue,min_revenue,avg_revenue,unique_package_types
0,64952,8130,413520.0,100.0,2.0,6.366548,4


In [ ]:
query_me_5 = """
-- Summarize advertising cost data:
-- Provides total and per-entry cost stats, plus diversity of ad sources and targeting.

SELECT
  SUM(cost) AS total_cost,                                -- Total advertising spend
  MAX(cost) AS max_cost,                                  -- Highest single cost entry
  MIN(cost) AS min_cost,                                  -- Lowest single cost entry
  AVG(cost) AS avg_cost,                                  -- Average cost per entry
  COUNT(DISTINCT network) AS unique_networks,             -- Number of distinct ad networks used
  COUNT(DISTINCT country) AS unique_countries,            -- Number of countries targeted in ad spend
  COUNT(DISTINCT platform) AS unique_platforms            -- Number of platforms involved in ad campaigns
FROM `case_db.q1_table_cost`;

"""

In [ ]:
 df_me_5 = conn.query(query_me_5).to_dataframe()

In [ ]:
df_me_5

,total_cost,max_cost,min_cost,avg_cost,unique_networks,unique_countries,unique_platforms
0,950731.0,14357.0,0.0,700.61238,5,5,2


# Magnitude Exploration

#### What?
We explore the **scale and distribution** of key performance indicators across dimensions such as:
- Installs by platform, network, and country
- Sessions by platform
- Revenue by platform and package type
- Advertising cost by network, country, and platform

This helps us understand the **relative contribution** and **dominance** of different categories.

---

#### Why?
Magnitude analysis allows us to:
- Identify **high-impact segments** (e.g., most valuable platforms or countries)
- Compare performance across channels to guide **budget allocation and optimization**
- Spot **disproportionate trends**, such as high cost but low return in certain areas

By examining aggregated values (e.g., total installs, total revenue, total cost), we uncover where most of the business activity is concentrated.

---

#### How?
We use SQL aggregation functions (`SUM`, `COUNT`, `AVG`, `MIN`, `MAX`) grouped by categorical dimensions.

In [ ]:
query_mae_1 = """
-- Generate install counts grouped by platform, network, and country:
-- Combines results into a single unified format for comparison across categories.

WITH installs_by_platform AS (
  SELECT
    'platform' AS category_type,                            -- Label to indicate this grouping is by platform
    platform AS category_value,                             -- Platform name (e.g., Android, iOS)
    COUNT(DISTINCT user_id) AS installs                     -- Unique user installs for each platform
  FROM `case_db.q1_table_install`
  GROUP BY platform
),

installs_by_network AS (
  SELECT
    'network' AS category_type,                             -- Label to indicate this grouping is by ad network
    network AS category_value,                              -- Ad network name
    COUNT(DISTINCT user_id) AS installs                     -- Unique user installs for each network
  FROM `case_db.q1_table_install`
  GROUP BY network
),

installs_by_country AS (
  SELECT
    'country' AS category_type,                             -- Label to indicate this grouping is by country
    country AS category_value,                              -- Country code or name
    COUNT(DISTINCT user_id) AS installs                     -- Unique user installs for each country
  FROM `case_db.q1_table_install`
  GROUP BY country
)

-- Combine all grouped install data into a single result set for easy comparison
SELECT * FROM (
  SELECT * FROM installs_by_platform
  UNION ALL
  SELECT * FROM installs_by_network
  UNION ALL
  SELECT * FROM installs_by_country
)
ORDER BY category_type, installs DESC;                       -- Sort by category, then by install count (descending)

"""

In [ ]:
 df_mae_1 = conn.query(query_mae_1).to_dataframe()

In [ ]:
df_mae_1

,category_type,category_value,installs
0,country,Mercury,87198
1,country,Pluton,59513
2,country,Venus,56136
3,country,Saturn,7278
4,country,Uranus,4957
5,network,Buzz,122093
6,network,Organic,58440
7,network,Woody,24650
8,network,Sid,7109
9,network,Jessie,2790


In [ ]:
query_mae_2 = """
-- Analyze session metrics grouped by platform:
-- Provides session volume and time-based engagement statistics per platform.

SELECT
  platform,                                                -- Platform identifier (e.g., Android, iOS)
  COUNT(*) AS session_count,                               -- Total number of sessions on the platform
  SUM(time_spent) AS total_time_spent,                     -- Cumulative time spent across sessions
  AVG(time_spent) AS avg_time_spent,                       -- Average time per session
  MAX(time_spent) AS max_time_spent,                       -- Longest individual session duration
  MIN(time_spent) AS min_time_spent                        -- Shortest individual session duration
FROM `case_db.q1_table_session`
GROUP BY platform;

"""

In [ ]:
 df_mae_2 = conn.query(query_mae_2).to_dataframe()

In [ ]:
df_mae_2

,platform,session_count,total_time_spent,avg_time_spent,max_time_spent,min_time_spent
0,None,8,83,10.375000,30,0
1,ios,168267028,4829211613,28.699691,155,0
2,android,129091821,3732232462,28.911456,79,0
3,andrgid,1,30,30.000000,30,30


In [ ]:
query_mae_3 = """
-- Analyze revenue performance by platform and package type:
-- Provides total and per-purchase revenue metrics for each combination.

SELECT
  platform,                                                -- Platform name (e.g., Android, iOS)
  package_type,                                            -- Type of purchase package (e.g., subscription, bundle)
  SUM(CAST(revenue AS FLOAT64)) AS total_revenue,          -- Total revenue generated by this combination
  AVG(CAST(revenue AS FLOAT64)) AS avg_revenue,            -- Average revenue per transaction
  MIN(CAST(revenue AS FLOAT64)) AS min_revenue,            -- Smallest transaction value
  MAX(CAST(revenue AS FLOAT64)) AS max_revenue             -- Largest transaction value
FROM `case_db.q1_table_revenue`
GROUP BY platform, package_type;

"""

In [ ]:
 df_mae_3 = conn.query(query_mae_3).to_dataframe()

In [ ]:
df_mae_3

,platform,package_type,total_revenue,avg_revenue,min_revenue,max_revenue
0,android,awesome_packs,37408.0,3.805106,2.0,100.0
1,ios,awesome_packs,109681.0,4.286256,2.0,100.0
2,ios,lovely_packs,181920.0,10.153486,2.0,100.0
3,android,lovely_packs,55689.0,8.632615,2.0,100.0
4,ios,pretty_packs,19863.0,5.512906,3.0,50.0
5,android,pretty_packs,8129.0,5.518669,3.0,100.0
6,ios,cool_packs,630.0,9.545455,5.0,10.0
7,android,cool_packs,200.0,9.090909,5.0,10.0


In [ ]:
query_mae_4 = """
-- Aggregate advertising cost metrics by different dimensions (network, country, platform):
-- Produces a unified view with cost summaries across each category type for comparison.

-- Cost aggregated by ad network
SELECT 'network' AS category_type,                           -- Identifies the grouping dimension
       network AS category_value,                            -- Specific ad network
       SUM(cost) AS total_cost,                              -- Total spend per network
       AVG(cost) AS avg_cost,                                -- Average spend per entry
       MIN(cost) AS min_cost,                                -- Minimum recorded cost
       MAX(cost) AS max_cost                                 -- Maximum recorded cost
FROM `case_db.q1_table_cost`
GROUP BY network

UNION ALL

-- Cost aggregated by country
SELECT 'country' AS category_type,
       country AS category_value,
       SUM(cost),
       AVG(cost),
       MIN(cost),
       MAX(cost)
FROM `case_db.q1_table_cost`
GROUP BY country

UNION ALL

-- Cost aggregated by platform
SELECT 'platform' AS category_type,
       platform AS category_value,
       SUM(cost),
       AVG(cost),
       MIN(cost),
       MAX(cost)
FROM `case_db.q1_table_cost`
GROUP BY platform;


"""

In [ ]:
 df_mae_4 = conn.query(query_mae_4).to_dataframe()

In [ ]:
df_mae_4

,category_type,category_value,total_cost,avg_cost,min_cost,max_cost
0,country,Venus,280072.0,936.695652,0.0,9044.0
1,country,Pluton,63558.0,243.517241,0.0,4053.0
2,country,Saturn,974.0,3.943320,0.0,275.0
3,country,Uranus,699.0,2.741176,0.0,117.0
4,country,Mercury,605428.0,2052.298305,0.0,14357.0
5,network,Sid,67168.0,283.409283,0.0,1843.0
6,network,Buzz,670052.0,2161.458065,0.0,14357.0
7,network,Woody,190834.0,621.609121,0.0,7828.0
8,network,Jessie,22677.0,117.497409,0.0,1014.0
9,network,Organic,0.0,0.000000,0.0,0.0
